In [1]:
from IMPORTS import *

c:\Python311\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Python311\Lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and 

In [2]:
Embedding_name="ft"
Dataset="ATE"

In [3]:
seed_value = 18
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [4]:
ft = fasttext.load_model('WordEmbeddings/cc.en.300.bin')

# Data Extraction and Preprocessing

Reading ATE json files

In [5]:
train = None
test = None
val = None
with open(f"Data\processed\{Dataset}_train.json") as f1:
    train = json.load(f1)

with open(f"Data\processed\{Dataset}_test.json") as f2:
    test = json.load(f2)

with open(f"Data\processed\{Dataset}_val.json") as f3:
    val = json.load(f3)



processing into lists

In [6]:
X_train,Y_train = create_train_test_val(train)
X_test,Y_test = create_train_test_val(test)
X_val,Y_val = create_train_test_val(val)

Tokenizing the ATE Dataset

In [7]:
X_train_tokenized, toke = TokenCreator(X_train,1)
X_test_tokenized = TokenCreator(X_test,0,tokenizer = toke)
X_val_tokenized = TokenCreator(X_val,0,tokenizer = toke)

Y_train_tokenized,toke2 = TokenCreator(Y_train,0)
Y_test_tokenized = TokenCreator(Y_test,0,tokenizer = toke2)
Y_val_tokenized = TokenCreator(Y_val,0,tokenizer = toke2)

Vocabulary of ATE

In [8]:
st1 = find_vocab(X_train_tokenized)
vocab_size  = len(st1) + 1 
vocab_size

2406

In [9]:
max_len=0

for i in X_train_tokenized:
    max_len = max(max_len,len(i))

max_len

78

Padding ATE

In [10]:
X_padded_train = pad_sequences(X_train_tokenized, maxlen=max_len, padding="pre", truncating="post")
Y_padded_train = pad_sequences(Y_train_tokenized, maxlen=max_len, padding="pre", truncating="post")

X_padded_test = pad_sequences(X_test_tokenized, maxlen=max_len, padding="pre", truncating="post")
Y_padded_test = pad_sequences(Y_test_tokenized, maxlen=max_len, padding="pre", truncating="post")

X_padded_val = pad_sequences(X_val_tokenized, maxlen=max_len, padding="pre", truncating="post")
Y_padded_val = pad_sequences(Y_val_tokenized, maxlen=max_len, padding="pre", truncating="post")       

One hot encoding the ATE labels

In [11]:
Y_padded_train = to_categorical(Y_padded_train)
Y_padded_test = to_categorical(Y_padded_test)
Y_padded_val = to_categorical(Y_padded_val)

Assigning **ATE FastText** mapping

In [12]:
weights = np.zeros((vocab_size, 300))
mapping = toke.word_index 
mapping

{'#UK': 1,
 'the': 2,
 '.': 3,
 ',': 4,
 'and': 5,
 'it': 6,
 'to': 7,
 'i': 8,
 'is': 9,
 'a': 10,
 'of': 11,
 'for': 12,
 'with': 13,
 'that': 14,
 'my': 15,
 'you': 16,
 'this': 17,
 'was': 18,
 'in': 19,
 'not': 20,
 'on': 21,
 'very': 22,
 'but': 23,
 'computer': 24,
 'use': 25,
 'have': 26,
 'great': 27,
 'they': 28,
 'laptop': 29,
 "n't": 30,
 'so': 31,
 'has': 32,
 'are': 33,
 'battery': 34,
 'had': 35,
 'screen': 36,
 ')': 37,
 'be': 38,
 'as': 39,
 'all': 40,
 '!': 41,
 '(': 42,
 "'s": 43,
 'also': 44,
 'me': 45,
 'just': 46,
 'easy': 47,
 'when': 48,
 'at': 49,
 'or': 50,
 'only': 51,
 'which': 52,
 'up': 53,
 'life': 54,
 'can': 55,
 'windows': 56,
 'than': 57,
 'out': 58,
 'if': 59,
 'do': 60,
 'after': 61,
 'more': 62,
 'keyboard': 63,
 'would': 64,
 'drive': 65,
 'hard': 66,
 'price': 67,
 'from': 68,
 'well': 69,
 'because': 70,
 'good': 71,
 'one': 72,
 'an': 73,
 'like': 74,
 'mac': 75,
 'about': 76,
 'quality': 77,
 'its': 78,
 'love': 79,
 'get': 80,
 'no': 81,
 'so

Fetching weights from pre trained FastText model (ATE)

In [13]:
for word, index in mapping.items():
    if word in ft:
        weights[index:] = ft[word]

In [14]:
X_train= X_padded_train
Y_train = Y_padded_train
X_test = X_padded_test
Y_test = Y_padded_test
X_val = X_padded_val
Y_val = Y_padded_val
weights = weights
word_token = toke
tag_token = toke2

In [15]:
Y_shape= Y_padded_train.shape[2]

# Models

## RNN

In [16]:
model_name="RNN"

### Training

In [17]:
rnn_model = Sequential()
rnn_model.add(Embedding(input_dim = vocab_size,output_dim = 300,input_length  = max_len,weights = [weights],trainable = False))
rnn_model.add(SimpleRNN(64, return_sequences=True))
rnn_model.add(TimeDistributed(Dense(Y_shape, activation='softmax')))

Compiling the rnn FastText Model

In [18]:
rnn_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc',Precision(),Recall()])

In [19]:
rnn_f1_callback = F1ScoreCallback(val_data=(X_val, Y_val),data=(X_train, Y_train))


In [20]:
rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 78, 300)           721800    
                                                                 
 simple_rnn (SimpleRNN)      (None, 78, 64)            23360     
                                                                 
 time_distributed (TimeDist  (None, 78, 4)             260       
 ributed)                                                        
                                                                 
Total params: 745420 (2.84 MB)
Trainable params: 23620 (92.27 KB)
Non-trainable params: 721800 (2.75 MB)
_________________________________________________________________


Fitting the rnn model

In [21]:
rnn_training = rnn_model.fit(X_train, Y_train, batch_size=128, epochs=15, validation_data=(X_val, Y_val), callbacks=[rnn_f1_callback])

Epoch 1/15




KeyboardInterrupt: 

Plotting the graph

In [ ]:
draw_loss_graph(rnn_training,model_name, rnn_f1_callback.train_f1_scores, rnn_f1_callback.val_f1_scores)
save_model(rnn_model,model_name,Embedding_name,Dataset)

### Testing

In [ ]:
loaded_model = load_model(model_name,Embedding_name,Dataset)

Testing accuracy

In [ ]:
accuracy = loaded_model.evaluate(X_test, Y_test, verbose = 1)
print(accuracy[1])

Testing F1 Score

In [ ]:
Y_pred = loaded_model.predict(X_test)
real = get_real(Y_test)
pred = get_pred(Y_pred)

true_labels_flat = [label for sublist in real for label in sublist]
predicted_labels_flat = [label for sublist in pred for label in sublist]

# Compute F1 score
f1 = f1_score(true_labels_flat, predicted_labels_flat, average='macro')

print("Test F1 Score:", f1)

## GRU

In [ ]:
model_name="GRU"

### Training

In [ ]:
gru_model = Sequential()
gru_model.add(Embedding(input_dim = vocab_size,output_dim = 300,input_length  = max_len,weights = [weights],trainable = False))
gru_model.add(GRU(64, return_sequences=True))
gru_model.add(TimeDistributed(Dense(Y_train.shape[2], activation='softmax')))

compiling 

In [ ]:
gru_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc',Precision(),Recall()])

In [ ]:
gru_f1_callback = F1ScoreCallback(val_data=(X_val, Y_val),data=(X_train, Y_train))

In [ ]:
gru_model.summary()

Fitting GRU

In [ ]:
gru_training = gru_model.fit(X_train, Y_train, batch_size=128, epochs=15, validation_data=(X_val, Y_val), callbacks=[gru_f1_callback])

Plotting Graph

In [ ]:
draw_loss_graph(gru_training,model_name, gru_f1_callback.train_f1_scores, gru_f1_callback.val_f1_scores)
save_model(gru_model,model_name,Embedding_name,Dataset)

### Testing

In [ ]:
loaded_model = load_model(model_name,Embedding_name,Dataset)

Testing Accuracy

In [ ]:
accuracy = loaded_model.evaluate(X_test, Y_test, verbose = 1)
print(accuracy[1])

Testing F1 Score

In [ ]:
Y_pred = loaded_model.predict(X_test)
real = get_real(Y_test)
pred = get_pred(Y_pred)

true_labels_flat = [label for sublist in real for label in sublist]
predicted_labels_flat = [label for sublist in pred for label in sublist]

# Compute F1 score
f1 = f1_score(true_labels_flat, predicted_labels_flat, average='macro')

print("Test F1 Score:", f1)

## LSTM

In [ ]:
model_name="LSTM"

### Training

In [ ]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim = vocab_size,output_dim = 300,input_length  = max_len,weights = [weights],trainable = False))
lstm_model.add(LSTM(64, return_sequences=True))
lstm_model.add(TimeDistributed(Dense(Y_train.shape[2], activation='softmax')))# Lstm

Compiling LSTM

In [ ]:
lstm_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc',Precision(),Recall()])

In [ ]:
LSTM_f1_callback=F1ScoreCallback(val_data=(X_val, Y_val),data=(X_train, Y_train))

In [ ]:
lstm_model.summary()

Fitting LSTM

In [ ]:
lstm_training = lstm_model.fit(X_train, Y_train, batch_size=128, epochs=15, validation_data=(X_val, Y_val), callbacks=[LSTM_f1_callback])

Plotting Graph

In [ ]:
draw_loss_graph(lstm_training, "LSTM Model", LSTM_f1_callback.train_f1_scores, LSTM_f1_callback.val_f1_scores)
save_model(lstm_model, model_name, Embedding_name, Dataset)
loaded_model = load_model(model_name,Embedding_name,Dataset)

### Testing

In [ ]:
loaded_model = load_model(model_name,Embedding_name,Dataset)

Testing accuracy

In [ ]:
accuracy = loaded_model.evaluate(X_test, Y_test, verbose = 1)
print(accuracy[1])

Testing F1 Score

In [ ]:
Y_pred = loaded_model.predict(X_test)
real = get_real(Y_test)
pred = get_pred(Y_pred)

true_labels_flat = [label for sublist in real for label in sublist]
predicted_labels_flat = [label for sublist in pred for label in sublist]

# Compute F1 score
f1 = f1_score(true_labels_flat, predicted_labels_flat, average='macro')

print("Test F1 Score:", f1)

# BiLSTM

In [ ]:
model_name="BiLSTM"

Training

In [ ]:
input=Input(shape=(max_len,))
BiLSTM_CRF_embed=Embedding(input_dim=vocab_size, output_dim=300, input_length=max_len, weights=[weights], trainable=False)(input)
BiLSTM_CRF_bilstm=Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.1))(BiLSTM_CRF_embed)
BiLSTM_model=(TimeDistributed(Dense(Y_train.shape[2], activation='relu')))(BiLSTM_CRF_bilstm)

crf = CRF(Y_train.shape[2])
predicted_sequence, potentials, sequence_length, crf_kernel = crf(BiLSTM_model)

BiLSTM_model=Model(input,potentials)

Compiling the BiLSTM Model

In [ ]:
BiLSTM_model.compile(
        optimizer=AdamW(learning_rate=0.002),
        loss=SigmoidFocalCrossEntropy(),
        metrics=['accuracy']
    )

In [ ]:
BiLSTM_f1_callback = F1ScoreCallback(val_data=(X_val, Y_val),data=(X_train, Y_train))

In [ ]:
BiLSTM_model.summary()

Fitting the BiLSTM Model

In [ ]:
BiLSTM_training = BiLSTM_model.fit(X_train,Y_train,batch_size=128,epochs=15,validation_data=(X_val,Y_val),callbacks=[BiLSTM_f1_callback])

Plotting the graph

In [ ]:
draw_loss_graph(BiLSTM_training,model_name, BiLSTM_f1_callback.train_f1_scores, BiLSTM_f1_callback.val_f1_scores)
save_model(BiLSTM_model,model_name,Embedding_name,Dataset)

## Testing 

In [ ]:
loaded_model = load_model(model_name,Embedding_name,Dataset)

Testing accuracy

In [ ]:
accuracy = loaded_model.evaluate(X_test, Y_test, verbose = 1)
print(accuracy[1])

Testing F1 Score

In [ ]:
Y_pred = loaded_model.predict(X_test)
real = get_real(Y_test)
pred = get_pred(Y_pred)

true_labels_flat = [label for sublist in real for label in sublist]
predicted_labels_flat = [label for sublist in pred for label in sublist]

# Compute F1 score
f1 = f1_score(true_labels_flat, predicted_labels_flat, average='macro')

print("Test F1 Score:", f1)

In [ ]:
Y_train.shape

(906, 78, 4)

In [ ]:
def Build_bilstm(max_len,vocab_size,weights,Y_train_shape,units=128,recurrent_dropout=0.1,optimizer='adam'):
    input=Input(shape=(max_len,))
    BiLSTM_CRF_embed=Embedding(input_dim=vocab_size, output_dim=300, input_length=max_len, weights=[weights], trainable=False)(input)
    BiLSTM_CRF_bilstm=Bidirectional(LSTM(units=units, return_sequences=True, recurrent_dropout=recurrent_dropout))(BiLSTM_CRF_embed)
    BiLSTM_model=(TimeDistributed(Dense(Y_train_shape, activation='relu')))(BiLSTM_CRF_bilstm)
    
    crf = CRF(Y_train_shape)
    predicted_sequence, potentials, sequence_length, crf_kernel = crf(BiLSTM_model)

    BiLSTM_model=Model(input,potentials)

    BiLSTM_model.compile(
        optimizer=optimizer,
        loss=SigmoidFocalCrossEntropy(),
        metrics=['accuracy']
    )
    print(BiLSTM_model.summary())
    return BiLSTM_model

def grid_search(X_train,Y_train,max_len):
    keras_model = KerasClassifier(build_fn=Build_bilstm,epochs=20,verbose=3,max_len=max_len,vocab_size=vocab_size,weights=weights,Y_train_shape=Y_train.shape[2],units=128,recurrent_dropout=0.1,optimizer=AdamW(learning_rate=0.002))
    param_grid={
        'recurrent_dropout':[0.1,0.05],
        'units':[128,256],
        'optimizer':['SGD','adam','rmsprop']
    }

    grid_search = GridSearchCV(keras_model,param_grid,cv=5,n_jobs=-1,verbose=3)
    print(grid_search)
    # Y_train = get_real(Y_train)
    grid_search.fit(X_train,Y_train)

    return grid_search.best_params_['recurrent_dropout'],grid_search.best_params_['units'],grid_search.best_params_['optimizer']


GridSearchCV = grid_search(X_train,Y_train,max_len)
recurrent_dropout = GridSearchCV[0]
units = GridSearchCV[1]
optimizer = GridSearchCV[2]

print(f"Best recurrent_dropout: {recurrent_dropout}")
print(f"Best units: {units}")
print(f"Best optimizer: {optimizer}")

GridSearchCV(cv=5,
             estimator=KerasClassifier(Y_train_shape=4, build_fn=<function Build_bilstm at 0x00000265B69E1F80>, epochs=20, max_len=78, optimizer=<keras.src.optimizers.adamw.AdamW object at 0x00000265C92BD810>, recurrent_dropout=0.1, units=128, verbose=3, vocab_size=2406, weights=array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.08007812,  0.10498047,  0....
       ...,
       [ 0.27734375,  0.03540039, -0.19921875, ..., -0.14453125,
         0.13085938, -0.04125977],
       [-0.19335938, -0.19238281, -0.28710938, ..., -0.1953125 ,
         0.15136719, -0.16015625],
       [ 0.10546875, -0.26171875,  0.01348877, ..., -0.06689453,
        -0.32226562, -0.08691406]])),
             n_jobs=-1,
             param_grid={'optimizer': ['SGD', 'adam', 'rmsprop'],
                         'recurrent_dropout': [0.1, 0.0

ValueError: 
All the 60 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py", line 760, in fit
    self._fit(
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py", line 915, in _fit
    X,

In [ ]:
# Define a function for model building with hyperparameters
def build_model(hp):
    input_dim = vocab_size
    output_dim = 300
    units = hp.Choice('units', values=[64, 128, 256])
    recurrent_dropout = hp.Choice('recurrent_dropout', values=[0.1, 0.2, 0.01, 0.05])
    optimizer = hp.Choice('optimizer', values=[AdamW(learning_rate=0.002,weight_decay=0.002),Adam(learning_rate=0.001),RMSprop(),SGD()])

    input_layer = Input(shape=(max_len,))
    embed_layer = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_len, weights=[weights], trainable=False)(input_layer)
    bilstm_layer = Bidirectional(LSTM(units=units, return_sequences=True, recurrent_dropout=recurrent_dropout))(embed_layer)
    time_distributed_layer = TimeDistributed(Dense(Y_train.shape[2], activation='relu'))(bilstm_layer)

    crf = CRF(Y_train.shape[2])
    predicted_sequence, potentials, sequence_length, crf_kernel = crf(time_distributed_layer)

    model = Model(input_layer, potentials)

    model.compile(
        optimizer=optimizer,
        loss=SigmoidFocalCrossEntropy(),
        metrics=['accuracy']
    )

    return model

# Define the tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,  # adjust based on resources available
    executions_per_trial=1,
    directory='tuner_directory',
    project_name='bilstm_tuner'
)

# Search for the best hyperparameters
tuner.search(X_train, Y_train, epochs=20, validation_split=0.2, verbose=3)

# Get the best hyperparameters
best_hps = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
print("Best Hyperparameters:")
print(best_hps)


Reloading Tuner from tuner_directory\bilstm_tuner\tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
160               |70                |max_len
128               |128               |units
0.1               |0.05              |recurrent_dropout
adam              |rmsprop           |optimizer



Traceback (most recent call last):
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
     

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\tuner.py", l

In [ ]:

# BiLSTM_model = Build_bilstm(max_len,vocab_size,weights,Y_train.shape[2],units,recurrent_dropout,optimizer)
